# 简介

这是一款名为‘Construction Assistant城市建设法规标准小助手’的演示应用，它完美展示了如何利用LangChain的工具以及ERNIEBot的functional agent来汇聚专业知识。通过利用ERNIE Bot SDK的functional agent，我们可以根据对话的上下文以及用户提出的具体问题，让大型模型在回答问题时灵活选择是否采用检索增强方式，或是直接给出答案。这种设计思路不仅丰富了大模型的领域知识，同时也保留了大模型在领域知识之外的通用对话能力。

构建流程如下：

# 1. 导入第三方库

In [21]:
import os
from typing import Optional, List, Type, Dict
from pydantic import BaseModel, Field
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

from erniebot_agent.extensions.langchain.embeddings import ErnieEmbeddings
from erniebot_agent.tools.base import Tool, ToolParameterView
from erniebot_agent.agents.functional_agent import FunctionalAgent
from erniebot_agent.memory.whole_memory import WholeMemory
from erniebot_agent.chat_models.erniebot import ERNIEBot
from erniebot_agent.messages import AIMessage, HumanMessage, Message
import erniebot

# 2. 预处理

## 2.1 下载数据集

In [22]:
! wget https://paddlenlp.bj.bcebos.com/datasets/examples/construction_regulations.tar
! tar xvf construction_regulations.tar

--2023-11-15 12:41:08--  https://paddlenlp.bj.bcebos.com/datasets/examples/construction_regulations.tar
Resolving paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)... 36.110.192.178
Connecting to paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)|36.110.192.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1239040 (1.2M) [application/x-tar]
Saving to: ‘construction_regulations.tar.1’

construction_regula 100%[===================>]   1.18M  3.19MB/s    in 0.4s    

2023-11-15 12:41:09 (3.19 MB/s) - ‘construction_regulations.tar.1’ saved [1239040/1239040]

x construction_regulations/
x construction_regulations/城市管理执法办法.pdf
x construction_regulations/建筑工程设计招标投标管理办法.pdf
x construction_regulations/建筑业企业资质管理规定.pdf
x construction_regulations/城市照明管理规定.pdf
x construction_regulations/城市设计管理办法.pdf
x construction_regulations/建筑工程施工发包与承包计价管理办法.pdf
x construction_regulations/市政公用设施抗灾设防管理规定.pdf


## 2.2 配置 ERNIE BOT API

这里使用aistudio的Access_token, 申请地址请参考[accessToken](https://aistudio.baidu.com/index/accessToken)

In [23]:
erniebot.api_type = "aistudio"
aistudio_access_token = "your access token"
erniebot.access_token = aistudio_access_token

# 3. Tool 构建

##  3.1 构建Schema

In [25]:
class SearchToolInputView(ToolParameterView):
    query: str = Field(description='规章查询语句')
    retrieval_num: int = Field(description="检索结果数目", default=5)


class SearchResponseDocument(ToolParameterView):
    document: str = Field(description="和query相关的规章片段")
    filename: str = Field(description="规章名称")
    page_num: int = Field(description="规章页数")


class SearchToolOutputView(ToolParameterView):
    documents: List[SearchResponseDocument] = Field(description="检索结果，内容为住房和城乡建设部规章中和query相关的规章片段")

## 3.2 构建Faiss检索工具

In [16]:
class FaissSearchTool(Tool):
    description: str = "在住房和城乡建设部规章中寻找和query最相关的片段"
    input_type: Type[ToolParameterView] = SearchToolInputView
    ouptut_type: Type[ToolParameterView] = SearchToolOutputView

    def __init__(self, db):
        self.db = db

    async def __call__(self, query: str, retrieval_num: int = 5) -> Dict[str, float]:
        docs = db.similarity_search(query)
        retrieval_results = []
        for doc in docs:
            retrieval_results.append(
                dict(SearchResponseDocument(
                    document=doc.page_content,
                    filename=doc.metadata["source"],
                    page_num=doc.metadata["page"],
                ))
            )
        return {"documents": retrieval_results}

    @property
    def examples(self) -> List[Message]:
        return [
                HumanMessage("城乡建设部规章中描述的城市管理执法的执法主体是谁？"),
                AIMessage(
                    "",
                    function_call={
                        "name": self.tool_name,
                        "thoughts": f"这是一个住房和城乡建设部规章的问题，我们使用{self.tool_name}工具检索相关的信息，检索的query：'城市管理执法的执法主体'。",
                        "arguments": '{"query": "城市管理执法的执法主体", "retrieval_num": 3}',
                    },
                )
            ]

# 4. Construction Assistant
## 4.1 建索引库

In [17]:
faiss_name = "faiss_index"
embeddings = ErnieEmbeddings(aistudio_access_token=aistudio_access_token, 
                             chunk_size=16,
                             sleep_time=0.5)

In [18]:
if os.path.exists(faiss_name):
    db = FAISS.load_local(faiss_name, embeddings)
else:
    loader = PyPDFDirectoryLoader("construction_regulations")
    documents = loader.load()
    # 换成CharacterTextSplitter会报错，因为会有超长文本切分
    # text_splitter = CharacterTextSplitter(chunk_size=320, chunk_overlap=0)
    text_splitter = SpacyTextSplitter(pipeline='zh_core_web_sm', chunk_size=320, chunk_overlap=0)
    
    docs = text_splitter.split_documents(documents)
    db = FAISS.from_documents(docs, embeddings)
    db.save_local(faiss_name)

In [33]:
tool = FaissSearchTool(db=db)
res = await tool(query="城市管理执法主管部门的职责是什么？")
from pprint import pprint
pprint(res)

{'documents': [{'document': '第十条 城市管理执法主管部门依法相对集中行使行政处罚权的， '
                            '可以实施法律法规规定的与行政处罚权相关的行政强制措施。',
                'filename': 'construction_regulations/城市管理执法办法.pdf',
                'page_num': 2},
               {'document': '第六条 '
                            '城市管理执法主管部门应当加强城市管理法律法规规章的宣传普及工作，增强全民守法意识，共同维护城市管理秩序。\n'
                            '\n'
                            '  \n'
                            '\n'
                            '第七条 城市管理执法主管部门应当积极为公众监督城市管理执法活动提供条件。',
                'filename': 'construction_regulations/城市管理执法办法.pdf',
                'page_num': 1},
               {'document': '其他违反法律法规和本办法规定的。   第四十条 '
                            '非城市管理执法人员着城市管理执法制式服装的，城市管理执法主管部门应当予以纠正，依法追究法律责任。',
                'filename': 'construction_regulations/城市管理执法办法.pdf',
                'page_num': 10},
               {'document': '第三章 执法主体     第十三条 '
                            '城市管理执法主管部门按照权责清晰、事权统一、精简效能的原则设置执法队伍。   第十四条 直辖市、 '
                       

## 4.2 构建Construction Assistant

In [29]:
llm = ERNIEBot(model="ernie-bot-8k")
memory = WholeMemory()
agent = FunctionalAgent(llm=llm, tools=[tool], memory=memory)
query = "城乡建设部规章中，城市管理执法第三章，第十三条是什么？"
response = await agent.async_run(query)
messages = response.chat_history
for item in messages:
    print(item.to_dict())

[Run][Start] Agent <erniebot_agent.agents.functional_agent.FunctionalAgent object at 0x125531a10> starts running with input: 城乡建设部规章中，城市管理执法第三章，第十三条是什么？
[LLM][Start] LLM <erniebot_agent.chat_models.erniebot.ERNIEBot object at 0x125550610> starts running with input: [<erniebot_agent.messages.HumanMessage object at 0x1240cec10>]
[LLM][End] LLM <erniebot_agent.chat_models.erniebot.ERNIEBot object at 0x125550610> finished running with output: role: assistant, function_call: {'name': 'FaissSearchTool', 'thoughts': '用户想要查询城乡建设部规章中关于城市管理执法第三章，第十三条的内容。我可以使用FaissSearchTool工具来查询相关内容。', 'arguments': '{"query":"城市管理执法第三章，第十三条","retrieval_num":1}'}
[Tool][Start] Tool <__main__.FaissSearchTool object at 0x12430d950> starts running with input: 
{
  "query": "城市管理执法第三章，第十三条",
  "retrieval_num": 1
}
[Tool][End] Tool <__main__.FaissSearchTool object at 0x12430d950> finished running with output: 
{
  "documents": [
    {
      "document": "第十条 城市管理执法主管部门依法相对集中行使行政处罚权的， 可以实施法律法规规定的与行政处罚权相关的行政强制措施。",
     